# Gruppenoperationen

Mit `groupby` ist ein Prozess gemeint, der einen oder mehrere der folgenden Schritte umfasst:

* **Split** teilt die Daten in Gruppen nach bestimmten Kriterien auf
* **Apply** wendet eine Funktion unabhängig auf jede Gruppe an
* **Combine** kombiniert die Ergebnisse in einer Datenstruktur

In der ersten Phase des Prozesses werden die in einem pandas-Objekt enthaltenen Daten, sei es eine Series, ein DataFrame oder etwas anderes, in Gruppen aufgeteilt, die auf einem oder mehreren Schlüsseln basieren. Die Aufteilung wird auf einer bestimmten Achse eines Objekts durchgeführt. Ein DataFrame kann zum Beispiel nach seinen Zeilen (`axis=0`) oder seinen Spalten (`axis=1`) gruppiert werden. Danach wird auf jede Gruppe eine Funktion angewendet, die einen neuen Wert erzeugt. Schließlich werden die Ergebnisse all dieser Funktionsanwendungen in einem Ergebnisobjekt kombiniert. Die Form des Ergebnisobjekts hängt normalerweise davon ab, was mit den Daten gemacht wird.

Jeder Gruppierungsschlüssel kann viele Formen annehmen, und die Schlüssel müssen nicht alle vom gleichen Typ sein:
* Eine Liste oder ein Array von Werten, die die gleiche Länge wie die zu gruppierende Achse haben
* Ein Wert, der einen Spaltennamen in einem DataFrame angibt
* Ein Dict oder eine Series, die eine Entsprechung zwischen den Werten auf der Achse, die gruppiert wird, und den Gruppennamen darstellt
* Eine Funktion, die auf dem Achsenindex oder den einzelnen Beschriftungen im Index aufgerufen wird

> **Hinweis:**
> 
> Die drei letztgenannten Methoden sind Abkürzungen, um ein Array von Werten zu erzeugen, die für die Aufteilung des Objekts verwendet werden.

Macht euch keine Sorgen, wenn dies alles abstrakt erscheint. Im Laufe dieses Kapitels werde ich viele Beispiele für all diese Methoden geben. Für den Anfang hier ein kleiner Tabellendatensatz als DataFrame:

In [1]:
import pandas as pd

df = pd.DataFrame({'Title' : ['Jupyter Tutorial',
                              'Jupyter Tutorial',
                              'PyViz Tutorial',
                              None,
                              'Python Basics',
                              'Python Basics'],
                   'Language' : ['de', 'en', 'de',  None, 'de', 'en'],
                   '12/2021' : [19651,4722,2573,None,525,157],
                   '01/2022' : [30134,3497,4873,None,427,85],
                   '02/2022' : [33295,4009,3930,None,276,226]})

df

,Title,Language,12/2021,01/2022,02/2022
0,Jupyter Tutorial,de,19651.0,30134.0,33295.0
1,Jupyter Tutorial,en,4722.0,3497.0,4009.0
2,PyViz Tutorial,de,2573.0,4873.0,3930.0
3,None,None,NaN,NaN,NaN
4,Python Basics,de,525.0,427.0,276.0
5,Python Basics,en,157.0,85.0,226.0


Angenommen, ihr möchtet den Summe der Spalte `02/2022` anhand der Beschriftungen von `Title` berechnen. Es gibt mehrere Möglichkeiten, dies zu tun. Eine davon ist der Zugriff auf `02/2022` und der Aufruf von `groupby` mit der Spalte (einer Series) in `Title`:

In [2]:
grouped = df['02/2022'].groupby(df['Title'])

grouped

Diese `grouped`-Variable ist nun ein spezielles `SeriesGroupBy`-Objekt. Es hat noch nichts berechnet, außer einigen Zwischendaten über den Gruppenschlüssel `df['Title']`. Die Idee ist, dass dieses Objekt über alle Informationen verfügt, die benötigt werden, um eine Operation auf jede der Gruppen anzuwenden. Zur Berechnung der Gruppenmittelwerte können wir beispielsweise die Methode `sum` des `GroupBy`-Objekts aufrufen:

In [3]:
grouped.sum()

Title
Jupyter Tutorial    37304.0
PyViz Tutorial       3930.0
Python Basics         502.0
Name: 02/2022, dtype: float64

Später werde ich mehr darüber erklären, was passiert, wenn ihr `.sum()` aufruft. Wichtig ist hier, dass die Daten (eine Reihe) durch Aufteilung der Daten auf den Gruppenschlüssel aggregiert wurden, wodurch eine neue Reihe entsteht, die nun durch die eindeutigen Werte in der Spalte `Title` indiziert ist. Der resultierende Index ist `Title`, weil `groupby(df['Title']` dies tat.

Hätten wir stattdessen mehrere Arrays als Liste übergeben, würden wir etwas anderes erhalten:

In [4]:
sums = df['12/2021'].groupby([df['Language'], df['Title']]).sum()

sums

Language  Title           
de        Jupyter Tutorial    19651.0
          PyViz Tutorial       2573.0
          Python Basics         525.0
en        Jupyter Tutorial     4722.0
          Python Basics         157.0
Name: 12/2021, dtype: float64

Hier haben wir die Daten anhand von zwei Schlüsseln gruppiert, und die resultierende Reihe hat nun einen hierarchischen Index, der aus den beobachteten eindeutigen Schlüsselpaaren besteht:

In [5]:
sums.unstack()

Title,Jupyter Tutorial,PyViz Tutorial,Python Basics
Language,,,
de,19651.0,2573.0,525.0
en,4722.0,NaN,157.0


Häufig befinden sich die Gruppierungsinformationen in demselben DataFrame wie die Daten, die ihr bearbeiten möchtet. In diesem Fall könnt ihr Spaltennamen (egal ob es sich um Zeichenketten, Zahlen oder andere Python-Objekte handelt) als Gruppenschlüssel übergeben:

In [6]:
df.groupby('Title').sum()

,12/2021,01/2022,02/2022
Title,,,
Jupyter Tutorial,24373.0,33631.0,37304.0
PyViz Tutorial,2573.0,4873.0,3930.0
Python Basics,682.0,512.0,502.0


Hierbei fällt auf, dass das Ergebnis keine Spalte `Language` enthält. Da es sich bei `df['Language']` nicht um numerische Daten handelt, stört sie im Tabellenlayout und wird daher automatisch aus dem Ergebnis ausgeschlossen. Standardmäßig werden alle numerischen Spalten aggregiert.

In [7]:
df.groupby(['Title','Language']).sum()

12/2021  01/2022  02/2022
Title            Language                           
Jupyter Tutorial de        19651.0  30134.0  33295.0
                 en         4722.0   3497.0   4009.0
PyViz Tutorial   de         2573.0   4873.0   3930.0
Python Basics    de          525.0    427.0    276.0
                 en          157.0     85.0    226.0

Unabhängig vom Ziel der Verwendung von `groupby` ist eine allgemein nützliche `groupby`-Methode `size`, die eine Serie mit den Gruppengrößen zurückgibt:

In [8]:
df.groupby(['Language']).size()

Language
de    3
en    2
dtype: int64

> **Hinweis:**
> 
> Alle fehlenden Werte in einem Gruppenschlüssel werden standardmäßig aus dem Ergebnis ausgeschlossen. Dieses Verhalten kann deaktiviert werden, indem `dropna=False` an `groupby` übergeben wird:

In [9]:
df.groupby('Language', dropna=False).size()

Language
de     3
en     2
NaN    1
dtype: int64

In [10]:
df.groupby(['Title', 'Language'], dropna=False).size()

Title             Language
Jupyter Tutorial  de          1
                  en          1
PyViz Tutorial    de          1
Python Basics     de          1
                  en          1
NaN               NaN         1
dtype: int64